In [194]:
import scipy.io
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
#from keras import utils
import time
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
tf.keras.backend.clear_session()  # For easy reset of notebook state.
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split



In [36]:
matrixodb = scipy.io.loadmat('./data/P1_odb.mat')
matrixstd = scipy.io.loadmat('./data/P1_std.mat')
P1odb = pd.DataFrame.from_dict(matrixodb['E_odb'])
P1std = pd.DataFrame.from_dict(matrixstd['E_std'])
P1odb['trig'] = np.ones((150))*1
P1std['trig'] = np.ones((1050))*-1

df = pd.concat([P1odb, P1std])
df_shuffled = df.sample(frac=1, random_state=0)

In [186]:
input_layer = keras.Input(shape = (1600), name='main_input')
x     = layers.Conv1D(16, 8, padding='same', activation='relu')(input_layer)
x     = layers.Conv1D(32, 6, padding='same', activation='relu')(x)
x     = layers.Conv1D(8, 4, padding='same', activation='relu')(x)
x     = layers.Conv1D(4, 2, padding='same', activation='relu')(x)
x     = layers.GlobalAveragePooling2D()(x)
x     = layers.Dense(8)(x)
x     = layers.Dense(64)(x)
output = layers.Dense(1, activation='softmax')(x)

model = keras.Model(inputs=input_layer, outputs=output)

model.summary()

ValueError: Input 0 of layer conv1d is incompatible with the layer: : expected min_ndim=3, found ndim=2. Full shape received: (None, 1600)

In [38]:
#compiling the model
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=True)  # default params
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
def train_net(model):
  appX = []
  appy = []
  init = time.time()
  for file in files:
    X, Y, Trials, Flash = prepare_data(file)
    X_clean, y_clean = clean_data(X, Y, Flash)
    appX.append(X_clean)
    appy.append(y_clean)

  X = [subject for subject in appX]
  y = [subject for subject in appy]
  X_train, X_valid, y_train, y_valid = train_test_split(np.vstack(X), np.vstack(y), test_size=0.1, random_state=816)
  history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=30, epochs=75, verbose=1)
  end = time.time()
  print("time elapsed training is:", (end - init)/60, " minutes")  
  return history.history['accuracy'], history.history['val_accuracy'], history.history['loss'], history.history['val_loss']

In [28]:
acc, val_acc, loss, val_loss = train_net(model)

NameError: name 'files' is not defined

In [224]:
train, test = train_test_split(df, test_size = 0.002)

In [225]:
x_train = train.to_numpy()[:, 0:-1]
y_train = train.to_numpy()[:, -1]

x_test = test.to_numpy()[:, 0:-1]
y_test = test.to_numpy()[:, -1]

In [229]:
np.shape(x_test)

(3, 1600)

In [243]:
input_shape = (10 ,20)

model = Sequential()

model.add(keras.layers.Conv1D(filters=32, kernel_size=5, input_shape=x_train.shape))
model.add(keras.layers.MaxPooling1D(pool_size=2 ))
#model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1000, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))

In [253]:
model.compile(opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=30, epochs=75, verbose=2)

Epoch 1/75


ValueError: in user code:

    /home/bci/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/bci/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/bci/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/bci/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/bci/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/bci/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:788 ru

In [252]:
model_m = Sequential()
#model_m.add(keras.layers.Reshape((200, 16), input_shape=(input_shape,)))
model_m.add(keras.layers.Conv1D(100, 10, activation='relu', input_shape=x_train.shape))
model_m.add(keras.layers.Conv1D(100, 10, activation='relu'))
model_m.add(keras.layers.MaxPooling1D(3))
model_m.add(keras.layers.Conv1D(160, 10, activation='relu'))
model_m.add(keras.layers.Conv1D(160, 10, activation='relu'))
model_m.add(keras.layers.GlobalAveragePooling1D())
model_m.add(keras.layers.Dropout(0.5))
model_m.add(keras.layers.Dense(2, activation='softmax'))
print(model_m.summary())

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_18 (Conv1D)           (None, 1188, 100)         1600100   
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 1179, 100)         100100    
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 393, 100)          0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 384, 160)          160160    
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 375, 160)          256160    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 160)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 160)             